# Objetivos

Primeiro, algumas variáveis serão descartadas pelo acúmulo de importância (~90%) e pelo valor de IV.

Segundo, com o uso da validação cruzada: será avaliado o comportamento médio de algumas métricas calculadas nos modelos baselines como a curva sobre a curva precision e recall (AUCPR), Brier Score (BS) e log-loss; foi visto que a identificação de um melhor ponto de corte pode melhorar a métrica F1-score. Dessa forma, será calculado qual o melhor ponto de corte médio.

Terceiro, será escolhido qual o melhor conjunto de hiperparâmetros para os modelos Random Forest e XGBoost.

# Pacotes

In [21]:
#!pip install hyperopt

In [1]:
from deltalake import DeltaTable, write_deltalake
import mlflow
from mlflow.models import infer_signature
import matplotlib.pyplot as plt
import os
import pandas as pd
import numpy as np
import shap
import Funcoes
from sklearn.model_selection import cross_val_score, StratifiedKFold, train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, roc_auc_score, precision_score, recall_score, accuracy_score, average_precision_score, brier_score_loss, confusion_matrix, classification_report, ConfusionMatrixDisplay, precision_recall_curve, log_loss
import xgboost as xgb
from hyperopt import fmin, tpe, Trials, hp
from functools import partial

/home/hugo/Documents/Git_GitHub/Estudo_Cartao_Credito/vCartao_Credito/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Leitura da base v1 e filtro de variáveis

Variáveis com valor acumulado de importância em ~90% e pelo valor de IV.

In [2]:
# A chance de ser ou não churn não depende do sexo ser M ou F. Mas, ela pode ser uma proxy para comportamentos socioeconômicos como diferença de renda, ocupação em 
# atividades não remuneradas (múltiplas jornadas de trabalho) etc

# Estado civil e número de dependentes podem ser interpretados de forma semelhante. Ambas podem direcionar para um maior compromisso com a estabilidade financeira,
# pessoas com mais idade e, possivelmente, com principalidade de uso do cartão já definida.

dados = DeltaTable("../1.Variaveis/tmp/dados_pp_v1").to_pandas()
dados.drop(['__index_level_0__', 'Card_Category_Gold', 'Card_Category_Platinum', 'Card_Category_Silver', 
            'Marital_Status_Married', 'Marital_Status_Single', 'Marital_Status_Unknown', 'Gender_M',
            'Dependent_count'], axis=1, inplace=True)
dados.head()

,Customer_Age,Total_Relationship_Count,Months_Inactive_12_mon,Contacts_Count_12_mon,Total_Revolving_Bal,Total_Amt_Chng_Q4_Q1,Total_Ct_Chng_Q4_Q1,vfm,pmcc,Income_Category_1.< 40k,Income_Category_2. >= 40k & < 60k,Income_Category_3. >= 60k & < 80k,Income_Category_4. >= 80k & < 120k,Income_Category_5. >= 120k,Education_Level_v2_1.Uneducated,Education_Level_v2_2.High School,Education_Level_v2_3.Graduate,Education_Level_v2_4.Post-Graduate,Attrition_Flag,type
0,-0.784196,1.403132,-1.337898,0.498943,0.963894,0.282975,-0.328225,-0.175537,-0.421450,1,0,0,0,0,0,0,1,0,0,Treino
1,0.720070,-0.525933,0.641818,1.408428,-0.165769,-1.527806,-0.194304,-0.208685,-1.054789,0,0,0,1,0,0,0,1,0,0,Treino
2,1.346848,-0.525933,-0.348040,0.498943,0.864865,0.894171,0.056797,-0.571459,-0.686436,0,0,0,1,0,0,0,0,1,0,Treino
3,0.218648,-0.525933,0.641818,-1.320028,-0.412731,0.369637,0.851953,0.252749,2.406712,1,0,0,0,0,0,0,1,0,0,Treino
4,-2.539173,0.117089,1.631675,1.408428,-0.858972,0.346832,-1.144306,-0.064053,-0.071911,1,0,0,0,0,1,0,0,0,0,Treino


In [3]:
dados.duplicated().sum()

np.int64(0)

In [4]:
dados.shape

(10127, 20)

# Separação das bases

In [3]:
dados_treino = dados[dados.type == 'Treino'].drop(['type'], axis=1)
dados_val = dados[dados.type == 'Validacao'].drop(['type'], axis=1)
dados_teste = dados[dados.type == 'Teste'].drop(['type'], axis=1)

In [4]:
X_treino = dados_treino.drop(['Attrition_Flag'], axis=1)
y_treino = dados_treino['Attrition_Flag']

X_val = dados_val.drop(['Attrition_Flag'], axis=1)
y_val = dados_val['Attrition_Flag']

In [15]:
y_treino.value_counts(), y_val.value_counts(), dados_teste['Attrition_Flag'].value_counts()

(Attrition_Flag
 0    6140
 1    1175
 Name: count, dtype: int64,
 Attrition_Flag
 0    1084
 1     208
 Name: count, dtype: int64,
 Attrition_Flag
 0    1276
 1     244
 Name: count, dtype: int64)

In [16]:
# Será usado 5 folds de validação, para que a quantidade absoluta de churn seja equiparada a base de validação e teste (ver acima)
1175/5

235.0

# Validação cruzada

Para ter uma avaliação mais precisa dos modelos baselines, será usada a validação cruzada. É importante ter em mente que precisa-se de uma quantidade razoável da variável target para o estudo, além de se assemelhar com a proporção verdadeira na base completa (~16%).

Para calcular os escores, das diferentes métricas, foi usado o cross_val_score (https://scikit-learn.org/1.5/modules/cross_validation.html).

Para a escolha das métricas de estudo, pode-se escolher pela lista definida em 3.4.3.1 (https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter).

## Random Forest

In [22]:
# Construção dos folds de forma estratificada para garantir a mesma representatividade da target

SKF = StratifiedKFold(n_splits=5, shuffle=True, random_state=563)

#  Instancia a Random Forest

rf = RandomForestClassifier(random_state=123)

# Aplicação da validação cruzada e obtenção dos escores

auc_pr_rf = cross_val_score(estimator=rf, X=X_treino, y=y_treino, cv=SKF, scoring='average_precision')
bs_rf = cross_val_score(estimator=rf, X=X_treino, y=y_treino, cv=SKF, scoring='neg_brier_score')
log_loss_rf = cross_val_score(estimator=rf, X=X_treino, y=y_treino, cv=SKF, scoring='neg_log_loss')

No mlflow, os valores de AUCPR e BS na base de validação foram, respectivamente, 0.80 e 0.06. Comparado com o resultado médio das métricas abaixo, pode-se concluir que o modelo com os hiperparâmetros default está com performance boa.

In [24]:
print('Valor médio de AUCPR:', auc_pr_rf.mean())
print('Desvio padrão de AUCPR:', auc_pr_rf.std())

print('----------------------------------------')

print('Valor médio do BS:', bs_rf.mean()*(-1))
print('Desvio padrão do BS:', bs_rf.std())

print('----------------------------------------')

print('Valor médio da log-loss:', log_loss_rf.mean()*(-1))
print('Desvio padrão da log-loss:', log_loss_rf.std())

Valor médio de AUCPR: 0.8110720675084535
Desvio padrão de AUCPR: 0.018858549324494565
----------------------------------------
Valor médio do BS: 0.06355405331510594
Desvio padrão do BS: 0.0023251419277794854
----------------------------------------
Valor médio da log-loss: 0.23914582093998044
Desvio padrão da log-loss: 0.017892713962885703


## XGBoost

In [6]:
# Ao usar o XGBoost, as colunas não podem ter sinais de '>', '>=', '<' e '<='
X_treino_new = X_treino.rename(columns={'Income_Category_1.< 40k': 'Income_Category_1.40k',
                                        'Income_Category_2. >= 40k & < 60k': 'Income_Category_2.40k_60k',
                                        'Income_Category_3. >= 60k & < 80k': 'Income_Category_3.60k_80k',
                                        'Income_Category_4. >= 80k & < 120k': 'Income_Category_4.80k_120k',
                                        'Income_Category_5. >= 120k': 'Income_Category_5.120k'
                                        })

X_val_new = X_val.rename(columns={'Income_Category_1.< 40k': 'Income_Category_1.40k',
                                        'Income_Category_2. >= 40k & < 60k': 'Income_Category_2.40k_60k',
                                        'Income_Category_3. >= 60k & < 80k': 'Income_Category_3.60k_80k',
                                        'Income_Category_4. >= 80k & < 120k': 'Income_Category_4.80k_120k',
                                        'Income_Category_5. >= 120k': 'Income_Category_5.120k'
                                        })

In [27]:
# Construção dos folds de forma estratificada para garantir a mesma representatividade da target

SKF = StratifiedKFold(n_splits=5, shuffle=True, random_state=563)

#  Instancia a XGBoost

XGB = xgb.XGBClassifier(random_state=123)

# Aplicação da validação cruzada e obtenção dos escores

auc_pr_XGB = cross_val_score(estimator=XGB, X=X_treino_new, y=y_treino, cv=SKF, scoring='average_precision')
bs_XGB = cross_val_score(estimator=XGB, X=X_treino_new, y=y_treino, cv=SKF, scoring='neg_brier_score')
log_loss_XGB = cross_val_score(estimator=XGB, X=X_treino_new, y=y_treino, cv=SKF, scoring='neg_log_loss')

No mlflow, os valores de AUCPR e BS na base de validação foram, respectivamente, 0.85 e 0.05. Comparado com o resultado médio das métricas abaixo, pode-se concluir que o modelo com os hiperparâmetros default está com performance boa.

Em média os valores das métricas são melhores para o XGB, entretanto perdem um pouco em relação à variância que é um pouco maior (trade-off).

In [28]:
print('Valor médio de AUCPR:', auc_pr_XGB.mean())
print('Desvio padrão de AUCPR:', auc_pr_XGB.std())

print('----------------------------------------')

print('Valor médio do BS:', bs_XGB.mean()*(-1))
print('Desvio padrão do BS:', bs_XGB.std())

print('----------------------------------------')

print('Valor médio da log-loss:', log_loss_XGB.mean()*(-1))
print('Desvio padrão da log-loss:', log_loss_XGB.std())

Valor médio de AUCPR: 0.827286855808228
Desvio padrão de AUCPR: 0.01765843965399739
----------------------------------------
Valor médio do BS: 0.059690328309613225
Desvio padrão do BS: 0.004629364361334095
----------------------------------------
Valor médio da log-loss: 0.21693788939008582
Desvio padrão da log-loss: 0.01650408191605959


## Avaliação do melhor ponto de corte médio com Random Forest

In [29]:
pontos_rf = []
f1_rf = []

for i, (treino_index, teste_index) in enumerate(SKF.split(X_treino, y_treino)):
    
    # Marcação do fold
    
    print(f"Fold {i}")
    
    # Divisão das bases de treino e teste
    
    X_treino_aux, X_teste_aux = X_treino.iloc[treino_index,:], X_treino.iloc[teste_index, :]
    y_treino_aux, y_teste_aux = y_treino[treino_index], y_treino[teste_index]

    # Aplicação da RF
    
    rf.fit(X_treino_aux, y_treino_aux)

    # Avaliação do melhor ponto de corte com o uso da base de validação

    precision, recall, thresholds = precision_recall_curve(y_teste_aux, rf.predict_proba(X_teste_aux)[:,1])

    # Melhor threshold que fornece o melhor f1-score (média harmônica entre precisão (Positive Predicted Value ou PPV) e recall (TPR))

    f1_scores = 2 * (precision*recall) / (precision + recall)
    best_threshold_index = np.argmax(f1_scores)
    best_threshold = thresholds[best_threshold_index]
    best_f1 = f1_scores[best_threshold_index]
    
    pontos_rf.append(best_threshold)
    f1_rf.append(best_f1)

Fold 0
Fold 1
Fold 2
Fold 3
Fold 4


Em média, o melhor ponto de corte é 0.356, garantindo em média um f1-score de 0.747. No mlflow, observa-se que a métrica f1-score na base de validação ficou em aproximadamente 0.686 (com o uso do ponto de corte default em 0.5), ou seja, é possível ter um ganho com a otimização do ponto de corte.

In [30]:
pontos_rf, f1_rf

([np.float64(0.37),
  np.float64(0.34),
  np.float64(0.41),
  np.float64(0.33),
  np.float64(0.33)],
 [np.float64(0.726086956521739),
  np.float64(0.7695560253699788),
  np.float64(0.7527839643652562),
  np.float64(0.7418032786885246),
  np.float64(0.7479674796747968)])

In [36]:
np.mean(pontos_rf), np.std(pontos_rf), np.mean(f1_rf)

(np.float64(0.356),
 np.float64(0.03072458299147442),
 np.float64(0.747639540924059))

## Avaliação do melhor ponto de corte médio com XGBoost

In [41]:
pontos_XGB = []
f1_XGB = []

for i, (treino_index, teste_index) in enumerate(SKF.split(X_treino_new, y_treino)):
    
    # Marcação do fold
    
    print(f"Fold {i}")
    
    # Divisão das bases de treino e teste
    
    X_treino_aux, X_teste_aux = X_treino_new.iloc[treino_index,:], X_treino_new.iloc[teste_index, :]
    y_treino_aux, y_teste_aux = y_treino[treino_index], y_treino[teste_index]

    # Aplicação da RF
    
    XGB.fit(X_treino_aux, y_treino_aux)

    # Avaliação do melhor ponto de corte com o uso da base de validação

    precision, recall, thresholds = precision_recall_curve(y_teste_aux, XGB.predict_proba(X_teste_aux)[:,1])

    # Melhor threshold que fornece o melhor f1-score (média harmônica entre precisão (Positive Predicted Value ou PPV) e recall (TPR))

    f1_scores = 2 * (precision*recall) / (precision + recall)
    best_threshold_index = np.argmax(f1_scores)
    best_threshold = thresholds[best_threshold_index]
    best_f1 = f1_scores[best_threshold_index]
    
    pontos_XGB.append(best_threshold)
    f1_XGB.append(best_f1)

Fold 0
Fold 1
Fold 2
Fold 3
Fold 4


Em média, o melhor ponto de corte é 0.369, garantindo em média um f1-score de 0.755. No mlflow, observa-se que a métrica f1-score na base de validação ficou em aproximadamente 0.755 (com o uso do ponto de corte default em 0.5), ou seja, não houve alteração nessa métrica.

Note que a variabilidade (desvio padrão) dos pontos de corte é maior comparado com o modelo Random Forest. Além disso, o ponto de corte médio para a RF é bem próximo do ponto de corte médio do XGBoost e os dois modelos possuem f1-score médio próximos.

In [42]:
pontos_XGB, f1_XGB

([np.float32(0.43493217),
  np.float32(0.30468673),
  np.float32(0.25663266),
  np.float32(0.3706298),
  np.float32(0.48277158)],
 [np.float64(0.7175925925925927),
  np.float64(0.7748917748917749),
  np.float64(0.7505154639175258),
  np.float64(0.7583148558758315),
  np.float64(0.7770114942528736)])

In [43]:
np.mean(pontos_XGB), np.std(pontos_XGB), np.mean(f1_XGB)

(np.float32(0.3699306),
 np.float32(0.082525104),
 np.float64(0.7556652363061197))

# Conclusão 1

Dos resultados anteriores, percebe-se que a diferença é pequena (e praticamente desprezível) entre as métricas médias dos modelos ajustados. Dessa forma, pode-se optar por um modelo mais simples como é o caso da RF.

Como forma de recapitular conceitos chaves, o modelo RF faz parte dos métodos de bagging. Neste grupo, as principais características são: possibilidade de execução dos algoritmos de forma paralela e/ou distribuída; redução de variância e menor sensibilidade a overfitting.

O modelo XGBoost faz parte dos métodos de boosting, com as seguintes características: a execução dos algoritmos é feita de forma sequencial, com enfoque em melhorar as predições nos grupos em que há mais erros e assim não pode ser executado de forma paralela e/ou distribuída; redução de viés e sensibilidade a overfitting (uma vez que o foco está nas observações com as piores predições).

# Ajuste de hiperparâmetros

In [7]:
X_treino2, X_teste2, y_treino2, y_teste2 = train_test_split(X_treino_new, y_treino, test_size=.40, stratify=y_treino, random_state=1234)
y_treino2.value_counts(), y_teste2.value_counts()

(Attrition_Flag
 0    3684
 1     705
 Name: count, dtype: int64,
 Attrition_Flag
 0    2456
 1     470
 Name: count, dtype: int64)

Pode ser que em alguma etapa de seleção de hiperparâmetros, algumas métricas não sejam bem definidas como precisão e recall, pois as medidas de falso positivo e verdadeiro positivo são iguais a zero ou falso negativo e verdadeiro positivo são iguais a zero. Assim, essas métricas foram desconsideras na função objetivo e mapeadas somente no final, na base de validação final.

Abaixo, a função objetivo tem como alvo minimizar a função log-loss e utiliza-se a otimização bayesiana para alcançar o melhor conjunto de hiperparâmetros (https://hyperopt.github.io/hyperopt/). 

In [21]:
# Função objetivo sem a validação cruzada

def func_objetivo(parametros, expr, modelo, X_treino, X_teste, y_treino, y_teste):
    # função objetivo para "minimizar", mas dependendo da métrica de interesse, na realidade, é maximizar 
    # parâmetros é o espaço paramétrico a ser explorado
    # expr é uma string que representa o id do experimento que foi criado
    # modelo é uma string de qual modelo será rodado: Random Forest ou XGBoost
    # X_treino, X_teste, y_treino, y_teste são as bases de treino e teste 

    # O output é o valor do score a ser minimizado/maximizado
    
    with mlflow.start_run(nested = True, experiment_id=expr) as run:

        if modelo == 'RF':
            clf = RandomForestClassifier(**parametros) 
            clf.fit(X_treino, y_treino)
        elif modelo == 'XGB':
            clf = xgb.XGBClassifier(**parametros)
            clf.fit(X_treino, y_treino)
        
        #score = log_loss(y_teste, clf.predict_proba(X_teste)[:,1], normalize=False)
        score = log_loss(y_teste, clf.predict_proba(X_teste)[:,1], normalize=True)

        # Log de parâmetros e métricas 

        mlflow.log_params(clf.get_params())
        mlflow.log_metric('log_loss_val', score)
        mlflow.log_metric('aucpr_val', average_precision_score(y_teste, clf.predict_proba(X_teste)[:,1]))
        mlflow.log_metric('roc_auc_score_val', roc_auc_score(y_teste, clf.predict_proba(X_teste)[:,1]))
        #mlflow.log_metric('f1_score_val', f1_score(y_teste, clf.predict(X_teste)))
        #mlflow.log_metric('precision_score_val', precision_score(y_teste, clf.predict(X_teste)))
        #mlflow.log_metric('recall_score_val', recall_score(y_teste, clf.predict(X_teste)))
        
        signature = infer_signature(X_treino, clf.predict_proba(X_treino))
        mlflow.sklearn.log_model(clf, signature=signature, artifact_path='modelo')

    return score


# Função objetivo com validação cruzada
# A otimização será feita considerando o comportamento médio da log-loss nos folds de validação

def func_objetivo_CV(parametros, modelo, folds, expr, X, y):
    # função objetivo para "minimizar", mas dependendo da métrica de interesse, na realidade, é maximizar 
    # parametros é o espaço paramétrico a ser explorado
    # expr é uma string que representa o id do experimento que foi criado
    # modelo é uma string de qual modelo será rodado: Random Forest ou XGBoost
    # folds é um int que diz quantos folds de validação serão usados
    # X e y são as bases que serão aplicadas o cross-validation

    # O output é o valor do score a ser minimizado/maximizado
    
    with mlflow.start_run(nested = True, experiment_id=expr) as run:

        SKF = StratifiedKFold(n_splits = folds, shuffle=True, random_state=1234)

        if modelo == 'RF':
            clf = RandomForestClassifier(**parametros) 
            clf.fit(X, y)
        elif modelo == 'XGB':
            clf = xgb.XGBClassifier(**parametros)
            clf.fit(X, y)
        
        #score = cross_val_score(estimator = clf, X = X, y = y, cv = SKF, scoring='average_precision').mean()
        score = cross_val_score(estimator = clf, X = X, y = y, cv = SKF, scoring='neg_log_loss').mean()

        # Log de parâmetros e métricas

        mlflow.log_params(clf.get_params())
        mlflow.log_metric('average_precision_cv', cross_val_score(estimator = clf, X = X, y = y, cv = SKF, scoring='average_precision').mean())
        mlflow.log_metric('roc_auc_cv', cross_val_score(estimator = clf, X = X, y = y, cv = SKF, scoring='roc_auc').mean())
        mlflow.log_metric('neg_brier_score_cv', cross_val_score(estimator = clf, X = X, y = y, cv = SKF, scoring='neg_brier_score').mean())
        mlflow.log_metric('neg_log_loss_cv', cross_val_score(estimator = clf, X = X, y = y, cv = SKF, scoring='neg_log_loss').mean())
        
        signature = infer_signature(X, clf.predict_proba(X))
        mlflow.sklearn.log_model(clf, signature=signature, artifact_path='modelo')

    return -score

In [14]:
#experiment = mlflow.create_experiment(name = 'Modelos_Tunados',
#                                      artifact_location = 'Artf_Modelos_Tunados',
#                                      tags = {'Environment': 'Development', 'Version': '2.0.0'}
#                                      )

In [23]:
#experiment

'840998848624696013'

In [9]:
experiment = mlflow.set_experiment(experiment_id='840998848624696013')

In [10]:
experiment.experiment_id

'840998848624696013'

## Random Forest

### Ajuste de hiperparâmetros sem validação cruzada

Espaço hiperparamétrico com 6 dimensões: quantidade de árvores, profundidade máxima, quantidade mínima de observações na folha, quantidade mínima de amostras para divisão do nó, critério para a divisão do nó e o peso atribuído as amostras devido ao desbalanceamento.

In [17]:
if __name__ == '__main__':
    
    #test_imbalanced = [{0: len(y_treino2)/(2*np.bincount(y_treino2))[0], 1:len(y_teste2)/(2*np.bincount(y_teste2))[1]}, {0: 1, 1:1}]
    
    space = {
        "n_estimators": hp.choice('n_estimators', np.arange(10, 500, dtype=int)),
        "max_depth": hp.choice('max_depth', np.arange(10, 300, dtype=int)),
        "min_samples_leaf": hp.choice('min_samples_leaf', np.arange(200, 500, dtype=int)),
        "min_samples_split": hp.choice('min_samples_split', np.arange(200, 500, dtype=int)),
        "criterion": hp.choice("criterion", ['gini', 'entropy', 'log_loss']),
        "class_weight": hp.choice("class_weight", ['balanced', 'balanced_subsample', None]) 
    }
    
    with mlflow.start_run(run_name = 'Random_Forest1', experiment_id=experiment.experiment_id) as run:
        best_params = fmin(
            fn = partial(
                func_objetivo,
                expr = experiment.experiment_id,
                modelo = 'RF',
                X_treino = X_treino2,
                X_teste = X_teste2,
                y_treino = y_treino2,
                y_teste = y_teste2
            ),
            space = space,
            algo = tpe.suggest,
            max_evals = 25,
            trials = Trials(),
            timeout = 10
        )

        if best_params['criterion'] == 0:
            best_params['criterion'] = 'gini'
        elif best_params['criterion'] == 1:
            best_params['criterion'] = 'entropy'
        else:
            best_params['criterion'] = 'log_loss'
            

        if best_params['class_weight'] == 0:
            best_params['class_weight'] = 'balanced'
        elif best_params['class_weight'] == 1:
            best_params['class_weight'] = 'balanced_subsample'
        else:
            best_params['class_weight'] = None
        
        # Identificado o melhor conjunto de hiperparâmetros, treina o modelo com toda a base de treino e metrifica os escores na base de validação

        clf = RandomForestClassifier(**best_params)
        clf.fit(X_treino_new, y_treino)
                   
        mlflow.log_params(clf.get_params())
        mlflow.log_metric('aucpr_val', average_precision_score(y_val, clf.predict_proba(X_val_new)[:,1]))
        mlflow.log_metric('log_loss_val', log_loss(y_val, clf.predict_proba(X_val_new)[:,1], normalize=True))
        mlflow.log_metric('roc_auc_score_val', roc_auc_score(y_val, clf.predict_proba(X_val_new)[:,1]))
        mlflow.log_metric('f1_score_val', f1_score(y_val, clf.predict(X_val_new)))
        mlflow.log_metric('f1_score_val2', f1_score(y_val, clf.predict_proba(X_val_new)[:,1] > 0.356))
        mlflow.log_metric('precision_score_val', precision_score(y_val, clf.predict(X_val_new)))
        mlflow.log_metric('precision_score_val2', precision_score(y_val, clf.predict_proba(X_val_new)[:,1] > 0.356))
        mlflow.log_metric('recall_score_val', recall_score(y_val, clf.predict(X_val_new)))
        mlflow.log_metric('recall_score_val2', recall_score(y_val, clf.predict_proba(X_val_new)[:,1] > 0.356))
        mlflow.log_metric('brier_score', brier_score_loss(y_val, clf.predict_proba(X_val_new)[:,1]))

        signature = infer_signature(X_treino_new, clf.predict_proba(X_treino_new))
        mlflow.sklearn.log_model(clf, signature=signature, artifact_path='modelo')

  0%|          | 0/25 [00:00<?, ?trial/s, best loss=?]

/home/hugo/Documents/Git_GitHub/Estudo_Cartao_Credito/vCartao_Credito/lib/python3.12/site-packages/mlflow/types/utils.py:435: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(



  4%|▍         | 1/25 [00:04<01:37,  4.07s/trial, best loss: 0.5243437337037734]

/home/hugo/Documents/Git_GitHub/Estudo_Cartao_Credito/vCartao_Credito/lib/python3.12/site-packages/mlflow/types/utils.py:435: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(



  8%|▊         | 2/25 [00:08<01:34,  4.12s/trial, best loss: 0.3383480247955717]

/home/hugo/Documents/Git_GitHub/Estudo_Cartao_Credito/vCartao_Credito/lib/python3.12/site-packages/mlflow/types/utils.py:435: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(



 12%|█▏        | 3/25 [00:13<01:36,  4.37s/trial, best loss: 0.3383480247955717]


/home/hugo/Documents/Git_GitHub/Estudo_Cartao_Credito/vCartao_Credito/lib/python3.12/site-packages/mlflow/types/utils.py:435: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(


Espaço hiperparamétrico com 2 dimensões, para alterar o menos possível os hiperparâmetros default, com os quais já se havia encontrado bons resultados!

In [18]:
if __name__ == '__main__':
    
    #test_imbalanced = [{0: len(y_treino2)/(2*np.bincount(y_treino2))[0], 1:len(y_teste2)/(2*np.bincount(y_teste2))[1]}, {0: 1, 1:1}]
    
    space = {
        "n_estimators": hp.choice('n_estimators', np.arange(10, 500, dtype=int)),
        "max_depth": hp.choice('max_depth', np.arange(10, 300, dtype=int))
    }
    
    with mlflow.start_run(run_name = 'Random_Forest2', experiment_id=experiment.experiment_id) as run:
        best_params = fmin(
            fn = partial(
                func_objetivo,
                expr = experiment.experiment_id,
                modelo = 'RF',
                X_treino = X_treino2,
                X_teste = X_teste2,
                y_treino = y_treino2,
                y_teste = y_teste2
            ),
            space = space,
            algo = tpe.suggest,
            max_evals = 25,
            trials = Trials(),
            timeout = 10
        )
       
        # Identificado o melhor conjunto de hiperparâmetros, treina o modelo com toda a base de treino e metrifica os escores na base de validação

        clf = RandomForestClassifier(**best_params)
        clf.fit(X_treino_new, y_treino)
                   
        mlflow.log_params(clf.get_params())
        mlflow.log_metric('aucpr_val', average_precision_score(y_val, clf.predict_proba(X_val_new)[:,1]))
        mlflow.log_metric('log_loss_val', log_loss(y_val, clf.predict_proba(X_val_new)[:,1], normalize=True))
        mlflow.log_metric('roc_auc_score_val', roc_auc_score(y_val, clf.predict_proba(X_val_new)[:,1]))
        mlflow.log_metric('f1_score_val', f1_score(y_val, clf.predict(X_val_new)))
        mlflow.log_metric('f1_score_val2', f1_score(y_val, clf.predict_proba(X_val_new)[:,1] > 0.356))
        mlflow.log_metric('precision_score_val', precision_score(y_val, clf.predict(X_val_new)))
        mlflow.log_metric('precision_score_val2', precision_score(y_val, clf.predict_proba(X_val_new)[:,1] > 0.356))
        mlflow.log_metric('recall_score_val', recall_score(y_val, clf.predict(X_val_new)))
        mlflow.log_metric('recall_score_val2', recall_score(y_val, clf.predict_proba(X_val_new)[:,1] > 0.356))
        mlflow.log_metric('brier_score', brier_score_loss(y_val, clf.predict_proba(X_val_new)[:,1]))


        signature = infer_signature(X_treino_new, clf.predict_proba(X_treino_new))
        mlflow.sklearn.log_model(clf, signature=signature, artifact_path='modelo')

  0%|          | 0/25 [00:00<?, ?trial/s, best loss=?]

/home/hugo/Documents/Git_GitHub/Estudo_Cartao_Credito/vCartao_Credito/lib/python3.12/site-packages/mlflow/types/utils.py:435: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(



  4%|▍         | 1/25 [00:03<01:33,  3.91s/trial, best loss: 0.2952681827606543]

/home/hugo/Documents/Git_GitHub/Estudo_Cartao_Credito/vCartao_Credito/lib/python3.12/site-packages/mlflow/types/utils.py:435: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(



  8%|▊         | 2/25 [00:11<02:14,  5.87s/trial, best loss: 0.2254783429168717]


/home/hugo/Documents/Git_GitHub/Estudo_Cartao_Credito/vCartao_Credito/lib/python3.12/site-packages/mlflow/types/utils.py:435: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(


### Ajuste de hiperparâmetros com validação cruzada

In [24]:
if __name__ == '__main__':
    
    space = {
        "n_estimators": hp.choice('n_estimators', np.arange(10, 500, dtype=int)),
        "max_depth": hp.choice('max_depth', np.arange(10, 300, dtype=int)),
        "min_samples_leaf": hp.choice('min_samples_leaf', np.arange(200, 500, dtype=int)),
        "min_samples_split": hp.choice('min_samples_split', np.arange(200, 500, dtype=int)),
        "criterion": hp.choice("criterion", ['gini', 'entropy', 'log_loss']),
        "class_weight": hp.choice("class_weight", ['balanced', 'balanced_subsample', None]) 
    }
    
    with mlflow.start_run(run_name = 'Random_Forest_CV1', experiment_id=experiment.experiment_id) as run:
        best_params = fmin(
            fn = partial(
                func_objetivo_CV,
                expr = experiment.experiment_id,
                modelo = 'RF',
                X = X_treino_new,
                y = y_treino,
                folds = 5
            ),
            space = space,
            algo = tpe.suggest,
            max_evals = 25,
            trials = Trials(),
            timeout = 10
        )

        if best_params['criterion'] == 0:
            best_params['criterion'] = 'gini'
        elif best_params['criterion'] == 1:
            best_params['criterion'] = 'entropy'
        else:
            best_params['criterion'] = 'log_loss'
            

        if best_params['class_weight'] == 0:
            best_params['class_weight'] = 'balanced'
        elif best_params['class_weight'] == 1:
            best_params['class_weight'] = 'balanced_subsample'
        else:
            best_params['class_weight'] = None
        
        # Identificado o melhor conjunto de hiperparâmetros, treina o modelo com toda a base de treino e metrifica os escores na base de validação

        clf = RandomForestClassifier(**best_params)
        clf.fit(X_treino_new, y_treino)
                   
        mlflow.log_params(clf.get_params())
        mlflow.log_metric('aucpr_val', average_precision_score(y_val, clf.predict_proba(X_val_new)[:,1]))
        mlflow.log_metric('log_loss_val', log_loss(y_val, clf.predict_proba(X_val_new)[:,1], normalize=True))
        mlflow.log_metric('roc_auc_score_val', roc_auc_score(y_val, clf.predict_proba(X_val_new)[:,1]))
        mlflow.log_metric('f1_score_val', f1_score(y_val, clf.predict(X_val_new)))
        mlflow.log_metric('f1_score_val2', f1_score(y_val, clf.predict_proba(X_val_new)[:,1] > 0.356))
        mlflow.log_metric('precision_score_val', precision_score(y_val, clf.predict(X_val_new)))
        mlflow.log_metric('precision_score_val2', precision_score(y_val, clf.predict_proba(X_val_new)[:,1] > 0.356))
        mlflow.log_metric('recall_score_val', recall_score(y_val, clf.predict(X_val_new)))
        mlflow.log_metric('recall_score_val2', recall_score(y_val, clf.predict_proba(X_val_new)[:,1] > 0.356))
        mlflow.log_metric('brier_score', brier_score_loss(y_val, clf.predict_proba(X_val_new)[:,1]))

        signature = infer_signature(X_treino_new, clf.predict_proba(X_treino_new))
        mlflow.sklearn.log_model(clf, signature=signature, artifact_path='modelo')

  0%|          | 0/25 [00:00<?, ?trial/s, best loss=?]

/home/hugo/Documents/Git_GitHub/Estudo_Cartao_Credito/vCartao_Credito/lib/python3.12/site-packages/mlflow/types/utils.py:435: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(



  4%|▍         | 1/25 [00:10<04:10, 10.45s/trial, best loss: 0.49348483924992664]


/home/hugo/Documents/Git_GitHub/Estudo_Cartao_Credito/vCartao_Credito/lib/python3.12/site-packages/mlflow/types/utils.py:435: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(


In [26]:
if __name__ == '__main__':
    
    space = {
        "n_estimators": hp.choice('n_estimators', np.arange(10, 500, dtype=int)),
        "max_depth": hp.choice('max_depth', np.arange(10, 300, dtype=int))
    }
    
    with mlflow.start_run(run_name = 'Random_Forest_CV2', experiment_id=experiment.experiment_id) as run:
        best_params = fmin(
            fn = partial(
                func_objetivo_CV,
                expr = experiment.experiment_id,
                modelo = 'RF',
                X = X_treino_new,
                y = y_treino,
                folds = 5
            ),
            space = space,
            algo = tpe.suggest,
            max_evals = 25,
            trials = Trials(),
            timeout = 10
        )

       
        # Identificado o melhor conjunto de hiperparâmetros, treina o modelo com toda a base de treino e metrifica os escores na base de validação

        clf = RandomForestClassifier(**best_params)
        clf.fit(X_treino_new, y_treino)
                   
        mlflow.log_params(clf.get_params())
        mlflow.log_metric('aucpr_val', average_precision_score(y_val, clf.predict_proba(X_val_new)[:,1]))
        mlflow.log_metric('log_loss_val', log_loss(y_val, clf.predict_proba(X_val_new)[:,1], normalize=True))
        mlflow.log_metric('roc_auc_score_val', roc_auc_score(y_val, clf.predict_proba(X_val_new)[:,1]))
        mlflow.log_metric('f1_score_val', f1_score(y_val, clf.predict(X_val_new)))
        mlflow.log_metric('f1_score_val2', f1_score(y_val, clf.predict_proba(X_val_new)[:,1] > 0.356))
        mlflow.log_metric('precision_score_val', precision_score(y_val, clf.predict(X_val_new)))
        mlflow.log_metric('precision_score_val2', precision_score(y_val, clf.predict_proba(X_val_new)[:,1] > 0.356))
        mlflow.log_metric('recall_score_val', recall_score(y_val, clf.predict(X_val_new)))
        mlflow.log_metric('recall_score_val2', recall_score(y_val, clf.predict_proba(X_val_new)[:,1] > 0.356))
        mlflow.log_metric('brier_score', brier_score_loss(y_val, clf.predict_proba(X_val_new)[:,1]))

        signature = infer_signature(X_treino_new, clf.predict_proba(X_treino_new))
        mlflow.sklearn.log_model(clf, signature=signature, artifact_path='modelo')

  0%|          | 0/25 [00:00<?, ?trial/s, best loss=?]

/home/hugo/Documents/Git_GitHub/Estudo_Cartao_Credito/vCartao_Credito/lib/python3.12/site-packages/mlflow/types/utils.py:435: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(



  4%|▍         | 1/25 [00:17<07:08, 17.86s/trial, best loss: 0.27413191438820855]


/home/hugo/Documents/Git_GitHub/Estudo_Cartao_Credito/vCartao_Credito/lib/python3.12/site-packages/mlflow/types/utils.py:435: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(
